In [1]:
import numpy as np                        # For linear algebra
# import cvxpy as cp
import matplotlib.pyplot as plt            # For plots
np.random.seed(1)                          # Generate random seed
np.set_printoptions(precision=1)           # Set nice printing format

In [2]:
# System parameters dx/dt = Ax + Bu, y = Cx
A = np.array([[1, 0],
              [0, 1]])
B = np.array([[1],
              [1]])
C = np.array([[1, 0]])

# Initial state
x0 = np.array([[5],[5]])

# parameters
t = 5 #response time [0, t-1]
T_d = 20 #total number of data points

# Collect data
x_data = np.zeros((T_d+1,2))
u_data = np.zeros((T_d,1))
y_data = np.zeros((T_d,1))

for i in range(T_d):
    u_data[i] = np.random.randn(1)
    x_data[i+1] = A@x_data[i] + B@u_data[i]
    y_data[i] = C@x_data[i]

In [3]:
# Build Hankel matrices
def hankel_matrix(data, rows, cols):
    H = np.zeros((rows, cols))
    for i in range(rows):
        H[i,:] = data[i:i+cols,0]
    return H

# build according to paper "Formulas for Data-Driven Control: Stabilization, Optimality, and Robustness" section II
H_u = hankel_matrix(u_data, t, T_d - t + 1)
H_y = hankel_matrix(y_data, t, T_d - t + 1)

print("Hankel matrix of inputs H_u:")
print(H_u)
print("\nHankel matrix of outputs H_y:")
print(H_y)

Hankel matrix of inputs H_u:
[[ 1.62434536 -0.61175641 -0.52817175 -1.07296862  0.86540763 -2.3015387
   1.74481176 -0.7612069   0.3190391  -0.24937038  1.46210794 -2.06014071
  -0.3224172  -0.38405435  1.13376944 -1.09989127]
 [-0.61175641 -0.52817175 -1.07296862  0.86540763 -2.3015387   1.74481176
  -0.7612069   0.3190391  -0.24937038  1.46210794 -2.06014071 -0.3224172
  -0.38405435  1.13376944 -1.09989127 -0.17242821]
 [-0.52817175 -1.07296862  0.86540763 -2.3015387   1.74481176 -0.7612069
   0.3190391  -0.24937038  1.46210794 -2.06014071 -0.3224172  -0.38405435
   1.13376944 -1.09989127 -0.17242821 -0.87785842]
 [-1.07296862  0.86540763 -2.3015387   1.74481176 -0.7612069   0.3190391
  -0.24937038  1.46210794 -2.06014071 -0.3224172  -0.38405435  1.13376944
  -1.09989127 -0.17242821 -0.87785842  0.04221375]
 [ 0.86540763 -2.3015387   1.74481176 -0.7612069   0.3190391  -0.24937038
   1.46210794 -2.06014071 -0.3224172  -0.38405435  1.13376944 -1.09989127
  -0.17242821 -0.87785842  0.04

In [9]:
#define random init to get a random data and solve for g
x = np.zeros((t+1,2))
u = np.zeros((t,1))
y = np.zeros((t,1))

for i in range(t):
    u[i] = np.random.randn(1)
    x[i+1] = A@x[i] + B@u[i]
    y[i] = C@x[i]

# solve for g
g = np.linalg.pinv(np.vstack((H_u, H_y))) @ np.vstack((u, y))
print("Solution g:")
print(g)

Solution g:
[[-0.31014612]
 [-0.38069813]
 [-0.06074878]
 [-0.19136547]
 [ 0.08296833]
 [ 0.12171322]
 [ 0.09072108]
 [ 0.03655381]
 [-0.00082582]
 [-0.33472981]
 [-0.20684722]
 [ 0.05306094]
 [ 0.12579016]
 [-0.13348685]
 [-0.19483915]
 [-0.01188344]]
